# Lonboard Example

In [43]:
from pathlib import Path

import pandas as pd
import geopandas as gpd
import shapely
from palettable.colorbrewer.diverging import BrBG_10
from sidecar import Sidecar

from lonboard import Map, ScatterplotLayer
from lonboard.colormap import apply_continuous_cmap

### Create a geo data frame

In [44]:
local_path = Path("./assets/internet-speeds.parquet")
url = "https://ookla-open-data.s3.us-west-2.amazonaws.com/parquet/performance/type=mobile/year=2019/quarter=1/2019-01-01_performance_mobile_tiles.parquet"

# Try to find data locally
if local_path.exists():
    gdf = gpd.read_parquet(local_path)
else:
    columns = ["avg_d_kbps", "tile"]
    try:
      # Download meteorite data and save it as a data frame
      df = pd.read_parquet(url, columns = columns)
    except Exception as e:
      print(f"Error: {e}")

    # Create tiles and centroids
    tile_geometries = shapely.from_wkt(df["tile"])
    tile_centroids = shapely.centroid(tile_geometries)

    # Create a geo data frame and pass it to a Parquet file
    gdf = gpd.GeoDataFrame(df[["avg_d_kbps"]], geometry = tile_centroids, crs = "EPSG:4326")
    gdf.to_parquet(local_path)

In [45]:
gdf.head(100)

,avg_d_kbps,geometry
0,5983,POINT (-160.01862 70.63722)
1,3748,POINT (-160.04059 70.63357)
2,3364,POINT (-160.04059 70.63175)
3,2381,POINT (-160.0351 70.63357)
4,3047,POINT (-160.0351 70.63175)
...,...,...
95,749,POINT (-52.12738 70.67543)
96,4271,POINT (-52.12189 70.67361)
97,1021,POINT (-52.11639 70.67724)
98,789,POINT (-51.63849 70.80949)


### Create a scatterplot layer

In [46]:
# Create a scatterplot layer from the geo data frame
layer = ScatterplotLayer.from_geopandas(gdf)

# Configure the layer
layer.get_fill_color = [0, 0, 200, 200]
min_bound = 5000
max_bound = 50000
download_speed = gdf["avg_d_kbps"]
normalized_download_speed = (download_speed - min_bound) / (max_bound - min_bound)
normalized_download_speed
BrBG_10.mpl_colormap
layer.get_fill_color = apply_continuous_cmap(normalized_download_speed, BrBG_10, alpha = 0.7)
layer.get_radius = normalized_download_speed * 200
layer.radius_units = "meters"
layer.radius_min_pixels = 0.5

### Create a map

In [47]:
sidecar = Sidecar()

# Create a Lonboard map
m = Map(layer)

# with Sidecar:
# display(m)
m

Map(custom_attribution='', layers=(ScatterplotLayer(get_fill_color=arro3.core.ChunkedArray<FixedSizeList(Field…